In [0]:
CREATE OR REPLACE VIEW workspace.`03_gold`.game_performance_data AS
WITH underperformers AS (
  SELECT 
    dg.game_name,
    g.genre_name,
    pub.publisher_name,
    plat.platform_name,
    dd.full_date AS release_date,
    SUM(fs.global_sales) AS total_sales,
    MAX(fs.rating) AS rating,
    MAX(fs.want_to_play_score) AS hype_raw,
    MAX(fs.played_score) AS engagement_raw
  FROM workspace.`03_gold`.fact_game_stats fs
  JOIN workspace.`03_gold`.dim_game dg ON fs.game_key = dg.game_key
  JOIN workspace.`03_gold`.dim_genre g ON fs.genre_key = g.genre_key
  JOIN workspace.`03_gold`.dim_publisher pub ON fs.publisher_key = pub.publisher_key
  JOIN workspace.`03_gold`.dim_platform plat ON fs.platform_key = plat.platform_key
  JOIN workspace.`03_gold`.dim_date dd ON fs.release_date_key = dd.date_key
  WHERE fs.rating IS NOT NULL
    AND fs.want_to_play_score IS NOT NULL
    AND fs.played_score IS NOT NULL
  GROUP BY dg.game_name, g.genre_name, pub.publisher_name, plat.platform_name, dd.full_date
),
normalized AS (
  SELECT 
    game_name,
    genre_name,
    publisher_name,
    platform_name,
    release_date,
    total_sales,
    rating,
    hype_raw,
    engagement_raw,
    -- Normalize sales (0-100)
    100.0 * (total_sales - MIN(total_sales) OVER ()) / 
      NULLIF(MAX(total_sales) OVER () - MIN(total_sales) OVER (), 0) AS sales_norm,
    -- Normalize rating (0-100)
    100.0 * (rating - MIN(rating) OVER ()) / 
      NULLIF(MAX(rating) OVER () - MIN(rating) OVER (), 0) AS rating_norm,
    -- Normalize hype (0-100)
    100.0 * (hype_raw - MIN(hype_raw) OVER ()) / 
      NULLIF(MAX(hype_raw) OVER () - MIN(hype_raw) OVER (), 0) AS hype_norm,
    -- Normalize engagement (0-100)
    100.0 * (engagement_raw - MIN(engagement_raw) OVER ()) / 
      NULLIF(MAX(engagement_raw) OVER () - MIN(engagement_raw) OVER (), 0) AS engagement_norm
  FROM underperformers
)
SELECT 
  game_name,
  genre_name,
  publisher_name,
  platform_name,
  YEAR(release_date) AS release_year,
  ROUND(total_sales / 1000000, 2) AS total_sales_millions,
  ROUND(sales_norm, 1) AS commercial_score,
  ROUND(rating_norm, 1) AS quality_score,
  ROUND(hype_norm, 1) AS pre_launch_hype,
  ROUND(engagement_norm, 1) AS player_engagement,
  ROUND(rating_norm - sales_norm, 1) AS quality_gap
FROM normalized
ORDER BY quality_gap DESC